In [1]:
'''
    Plot overlapping features - TURING analysis
'''

import pandas as pd
import matplotlib.pyplot as plt
import scipy.io
import mne
import numpy as np
import mat73
import os.path as osp
from mne_connectivity.viz import plot_connectivity_circle
from mne.viz import circular_layout

from sklearn.preprocessing import MinMaxScaler, Normalizer
from scipy.io import savemat

In [2]:
## functions
def isSquare (m): return all (len (row) == len (m) for row in m)

def regions_bst_to_lobes( argument ):
    '''
        gather brainstorm labels in 5 lobes/areas
    '''
    switcher = {
        # gathers Left and Right in one single lobe
        # Frontal: includes both L & R
        'LF': 'F' ,
        'RF': 'F' ,
        # Temporal: includes both both L & R
        'LT': 'T' ,
        'RT': 'T' ,
        # Parietal
        'LP': 'P' ,
        'RP': 'P' ,
        # Occipital
        'LO': 'O' ,
        'RO': 'O',
        # NB
        'NB': 'NB',
        # Cer
        'Cer': 'Cer'
    }
    return switcher.get ( argument )

def Lobes_Partition( MatrixToBeDivided , idx_F  , idx_P , idx_T , idx_O , idx_NB, idx_Cer):
    SubMatrix_F_F = MatrixToBeDivided[ np.ix_ ( idx_F , idx_F ) ]
    SubMatrix_F_P = MatrixToBeDivided[ np.ix_ ( idx_F , idx_P ) ]
    SubMatrix_F_T = MatrixToBeDivided[ np.ix_ ( idx_F , idx_T ) ]
    SubMatrix_F_O = MatrixToBeDivided[ np.ix_ ( idx_F , idx_O ) ]
    SubMatrix_F_NB = MatrixToBeDivided[np.ix_(idx_F, idx_NB)]
    SubMatrix_F_Cer = MatrixToBeDivided[np.ix_(idx_F, idx_Cer)]

    SubMatrix_P_P = MatrixToBeDivided[ np.ix_ ( idx_P , idx_P ) ]
    SubMatrix_P_T = MatrixToBeDivided[ np.ix_ ( idx_P , idx_T ) ]
    SubMatrix_P_O = MatrixToBeDivided[ np.ix_ ( idx_P , idx_O ) ]
    SubMatrix_P_NB = MatrixToBeDivided[ np.ix_ ( idx_P , idx_NB ) ]
    SubMatrix_P_Cer = MatrixToBeDivided[np.ix_(idx_P, idx_Cer)]

    SubMatrix_T_T = MatrixToBeDivided[ np.ix_ ( idx_T , idx_T ) ]
    SubMatrix_T_O = MatrixToBeDivided[ np.ix_ ( idx_T , idx_O ) ]
    SubMatrix_T_NB = MatrixToBeDivided[np.ix_(idx_T, idx_NB)]
    SubMatrix_T_Cer = MatrixToBeDivided[np.ix_(idx_T, idx_Cer)]

    SubMatrix_O_O = MatrixToBeDivided[ np.ix_ ( idx_O , idx_O ) ]
    SubMatrix_O_NB = MatrixToBeDivided[np.ix_(idx_O, idx_NB)]
    SubMatrix_O_Cer = MatrixToBeDivided[np.ix_(idx_O, idx_Cer)]

    SubMatrix_NB_NB = MatrixToBeDivided[np.ix_(idx_NB, idx_NB)]
    SubMatrix_NB_Cer = MatrixToBeDivided[np.ix_(idx_NB, idx_Cer)]

    SubMatrix_Cer_Cer = MatrixToBeDivided[ np.ix_ ( idx_Cer , idx_Cer ) ]

    return SubMatrix_F_F , SubMatrix_F_P , SubMatrix_F_T , SubMatrix_F_O , SubMatrix_F_NB , SubMatrix_F_Cer, \
           SubMatrix_P_P , SubMatrix_P_T , SubMatrix_P_O , SubMatrix_P_NB , SubMatrix_P_Cer, \
           SubMatrix_T_T, SubMatrix_T_O, SubMatrix_T_NB, SubMatrix_T_Cer, \
           SubMatrix_O_O , SubMatrix_O_NB , SubMatrix_O_Cer , \
           SubMatrix_NB_NB, SubMatrix_NB_Cer, \
           SubMatrix_Cer_Cer

def plot_connectome_features_lobes_AAL(fig_path, db_path,fname_mat,
                                               idx_F_AAL, idx_P_AAL, idx_T_AAL, idx_O_AAL, idx_NB_AAL, idx_Cer_AAL,
                                               ROI_AAL_list, Region_AAL):

    data_dict = scipy.io.loadmat(db_path + fname_mat + '.mat')
    temp_sparse_matrix = data_dict['data']
    sparse_matrix = np.triu(temp_sparse_matrix) + np.tril(temp_sparse_matrix.T, 1)
    # scaler = MinMaxScaler() # to normalize the data from 0 to 1
    # scaler.fit(sparse_matrix)
    # sparse_matrix = scaler.transform(sparse_matrix)

    max = np.max(np.abs((sparse_matrix)))
    if max>0 :
        [SubMatrix_F_F, SubMatrix_F_P, SubMatrix_F_T, SubMatrix_F_O, SubMatrix_F_NB, SubMatrix_F_Cer, \
         SubMatrix_P_P, SubMatrix_P_T, SubMatrix_P_O, SubMatrix_P_NB, SubMatrix_P_Cer, \
         SubMatrix_T_T, SubMatrix_T_O, SubMatrix_T_NB, SubMatrix_T_Cer, \
         SubMatrix_O_O, SubMatrix_O_NB, SubMatrix_O_Cer, \
         SubMatrix_NB_NB, SubMatrix_NB_Cer, \
         SubMatrix_Cer_Cer] = Lobes_Partition(sparse_matrix, idx_F_AAL, idx_P_AAL, idx_T_AAL, idx_O_AAL, idx_NB_AAL,
                                              idx_Cer_AAL)

        temp1 = np.hstack((SubMatrix_F_F, SubMatrix_F_P, SubMatrix_F_T, SubMatrix_F_O, SubMatrix_F_NB, SubMatrix_F_Cer))
        temp2 = np.hstack(
            (np.transpose(SubMatrix_F_P), SubMatrix_P_P, SubMatrix_P_T, SubMatrix_P_O, SubMatrix_P_NB, SubMatrix_P_Cer))
        temp1b = np.vstack((temp1, temp2))
        temp3 = np.hstack((np.transpose(SubMatrix_F_T), np.transpose(SubMatrix_P_T), SubMatrix_T_T, SubMatrix_T_O,
                           SubMatrix_T_NB, SubMatrix_T_Cer))
        temp4 = np.hstack((np.transpose(SubMatrix_F_O), np.transpose(SubMatrix_P_O), np.transpose(SubMatrix_T_O),
                           SubMatrix_O_O, SubMatrix_O_NB, SubMatrix_O_Cer))
        temp3b = np.vstack((temp3, temp4))
        temp4b = np.vstack((temp1b, temp3b))
        temp5 = np.hstack((np.transpose(SubMatrix_F_NB), np.transpose(SubMatrix_P_NB), np.transpose(SubMatrix_T_NB),
                           np.transpose(SubMatrix_O_NB), SubMatrix_NB_NB, SubMatrix_NB_Cer))
        temp5b = np.vstack((temp4b, temp5))
        temp6 = np.hstack((np.transpose(SubMatrix_F_Cer), np.transpose(SubMatrix_P_Cer), np.transpose(SubMatrix_T_Cer),
                           np.transpose(SubMatrix_O_Cer), np.transpose(SubMatrix_NB_Cer), SubMatrix_Cer_Cer))
        temp6b = np.vstack((temp5b, temp6))
        sig_diff_rho_plot = temp6b

        ROI_AAL_list_F = [ROI_AAL_list[i] for i in idx_F_AAL]
        ROI_AAL_list_P = [ROI_AAL_list[i] for i in idx_P_AAL]
        ROI_AAL_list_T = [ROI_AAL_list[i] for i in idx_T_AAL]
        ROI_AAL_list_0 = [ROI_AAL_list[i] for i in idx_O_AAL]
        ROI_AAL_list_NB = [ROI_AAL_list[i] for i in idx_NB_AAL]
        ROI_AAL_list_Cer = [ROI_AAL_list[i] for i in idx_Cer_AAL]
        ROI_AAL_list_by_lobes = list(np.concatenate(
            (ROI_AAL_list_F, ROI_AAL_list_P, ROI_AAL_list_T, ROI_AAL_list_0, ROI_AAL_list_NB, ROI_AAL_list_Cer)))

        Region_AAL_colors = Region_AAL
        for i in idx_F_AAL:
            Region_AAL_colors[i] = 'firebrick'
        for i in idx_P_AAL:
            Region_AAL_colors[i] = 'darkorange'
        for i in idx_T_AAL:
            Region_AAL_colors[i] = 'darkolivegreen'
        for i in idx_O_AAL:
            Region_AAL_colors[i] = 'cadetblue'
        for i in idx_NB_AAL:
            Region_AAL_colors[i] = 'mediumpurple'
        for i in idx_Cer_AAL:
            Region_AAL_colors[i] = 'burlywood'

        Region_AAL_color_F = [Region_AAL_colors[i] for i in idx_F_AAL]
        Region_AAL_color_P = [Region_AAL_colors[i] for i in idx_P_AAL]
        Region_AAL_color_T = [Region_AAL_colors[i] for i in idx_T_AAL]
        Region_AAL_color_O = [Region_AAL_colors[i] for i in idx_O_AAL]
        Region_AAL_color_NB = [Region_AAL_colors[i] for i in idx_NB_AAL]
        Region_AAL_color_Cer = [Region_AAL_colors[i] for i in idx_Cer_AAL]
        Region_AAL_color_by_lobes = list(np.concatenate((Region_AAL_color_F, Region_AAL_color_P, Region_AAL_color_T,
                                                         Region_AAL_color_O, Region_AAL_color_NB, Region_AAL_color_Cer)))

        node_angles = circular_layout(
            ROI_AAL_list_by_lobes, ROI_AAL_list_by_lobes, start_pos=90,
        )

        fig_corr, ax = plt.subplots(figsize=(18,18), facecolor='white',
                              subplot_kw=dict(polar=True)) # 6,6
        plot_connectivity_circle(sig_diff_rho_plot, node_names=ROI_AAL_list_by_lobes,
                                 node_colors=Region_AAL_color_by_lobes,
                                 node_angles=node_angles,
                                 fontsize_names=18,
                                 colormap='Greys',
                                 facecolor='white', textcolor='black',
                                 node_edgecolor='white',
                                 vmin=0, vmax=max,
                                 linewidth=6,
                                 fig=fig_corr, ax=ax, show=False)

        fig_corr.savefig(
            fig_path + 'Connect_OverlapingFeatures_' + fname_mat + '.png',
            dpi=600, facecolor='white')
    else:
        print('Unexploitable data')

def Mask_Lobes_Partition_AAL(nb_ROIs, idx_F  , idx_P , idx_T , idx_O , idx_NB, idx_Cer):
    mask=np.zeros((nb_ROIs,nb_ROIs))

    mask[np.ix_(idx_F, idx_F)] = 1
    mask[ np.ix_ ( idx_F , idx_P ) ] = 2
    mask[ np.ix_ ( idx_P , idx_F ) ] = 2
    mask[np.ix_( idx_F, idx_T )] = 3
    mask[ np.ix_ ( idx_T , idx_F ) ] = 3
    mask[ np.ix_ ( idx_F , idx_O ) ] = 4
    mask[ np.ix_ ( idx_O , idx_F ) ] = 4
    mask[ np.ix_ ( idx_F , idx_NB ) ] = 5
    mask[ np.ix_ ( idx_NB , idx_F ) ] = 5
    mask[ np.ix_ ( idx_F , idx_Cer ) ] = 6
    mask[ np.ix_ ( idx_Cer , idx_F ) ] = 6


    mask[np.ix_(idx_P, idx_P)]=7
    mask[np.ix_(idx_P, idx_T)]=8
    mask[ np.ix_ ( idx_T , idx_P ) ] = 8
    mask[np.ix_(idx_P, idx_O)]= 9
    mask[ np.ix_ ( idx_O , idx_P ) ] = 9
    mask[np.ix_(idx_P, idx_NB)]= 10
    mask[ np.ix_ ( idx_NB , idx_P ) ] = 10
    mask[np.ix_(idx_P, idx_Cer)]= 11
    mask[ np.ix_ ( idx_Cer , idx_P ) ] = 11

    mask[np.ix_(idx_T, idx_T)]= 12
    mask[np.ix_(idx_T, idx_O)]= 13
    mask[ np.ix_ ( idx_O , idx_T ) ] = 13
    mask[np.ix_(idx_T, idx_NB)]= 14
    mask[ np.ix_ ( idx_NB , idx_T ) ] = 14
    mask[np.ix_(idx_T, idx_Cer)]= 15
    mask[ np.ix_ ( idx_Cer , idx_T ) ] = 15

    mask[np.ix_(idx_O, idx_O)]= 16
    mask[np.ix_(idx_O, idx_NB)]= 17
    mask[ np.ix_ ( idx_NB , idx_O ) ] = 17
    mask[np.ix_(idx_O, idx_Cer)]= 18
    mask[ np.ix_ ( idx_Cer , idx_O ) ] = 18

    mask[np.ix_(idx_NB, idx_NB)]= 19
    mask[np.ix_(idx_NB, idx_Cer)] = 20
    mask[np.ix_(idx_Cer, idx_NB)] = 20

    mask[np.ix_(idx_Cer, idx_Cer)] = 21

    mask = {"mask": mask}

    savemat(db_path + "mask_AAL.mat", mask)


    return mask

In [6]:
pwd

'/Users/dmytro/Documents/Projects/PROJECT_main/multiclass_meg_features_analysis/notebooks'

In [7]:
############################################# APPLICATION TO REAL DATA #############################################
root_path = '/Users/dmytro/Documents/Projects/PROJECT_main/multiclass_meg_features_analysis'
db_path = root_path + '/data/overlap_features'
fig_path = root_path + '/figures/overlap_features'

# AAL
ROI_AAL_list = ['Rectus_L', 'Olfactory_L', 'Frontal_Sup_Orb_L', 'Frontal_Med_Orb_L', 'Frontal_Mid_Orb_L',
                'Frontal_Inf_Orb_L', 'Frontal_Sup_L', 'Frontal_Mid_L', 'Frontal_Inf_Oper_L', 'Frontal_Inf_Tri_L',
                'Frontal_Sup_Medial_L', 'Supp_Motor_Area_L', 'Paracentral_Lobule_L', 'Precentral_L',
                'Rolandic_Oper_L', 'Postcentral_L',
                'Parietal_Sup_L',
                'Parietal_Inf_L',
                'SupraMarginal_L',
                'Angular_L',
                'Precuneus_L',
                'Occipital_Sup_L',
                'Occipital_Mid_L',
                'Occipital_Inf_L',
                'Calcarine_L',
                'Cuneus_L',
                'Lingual_L',
                'Fusiform_L',
                'Heschl_L',
                'Temporal_Sup_L',
                'Temporal_Mid_L',
                'Temporal_Inf_L',
                'Temporal_Pole_Sup_L',
                'Temporal_Pole_Mid_L',
                'ParaHippocampal_L',
                'Cingulum_Ant_L',
                'Cingulum_Mid_L',
                'Cingulum_Post_L',
                'Insula_L',
                'Rectus_R',
                'Olfactory_R',
                'Frontal_Sup_Orb_R',
                'Frontal_Med_Orb_R',
                'Frontal_Mid_Orb_R',
                'Frontal_Inf_Orb_R',
                'Frontal_Sup_R',
                'Frontal_Mid_R',
                'Frontal_Inf_Oper_R',
                'Frontal_Inf_Tri_R',
                'Frontal_Sup_Medial_R',
                'Supp_Motor_Area_R',
                'Paracentral_Lobule_R',
                'Precentral_R',
                'Rolandic_Oper_R',
                'Postcentral_R',
                'Parietal_Sup_R',
                'Parietal_Inf_R',
                'SupraMarginal_R',
                'Angular_R',
                'Precuneus_R',
                'Occipital_Sup_R',
                'Occipital_Mid_R',
                'Occipital_Inf_R',
                'Calcarine_R',
                'Cuneus_R',
                'Lingual_R',
                'Fusiform_R',
                'Heschl_R',
                'Temporal_Sup_R',
                'Temporal_Mid_R',
                'Temporal_Inf_R',
                'Temporal_Pole_Sup_R',
                'Temporal_Pole_Mid_R',
                'ParaHippocampal_R',
                'Cingulum_Ant_R',
                'Cingulum_Mid_R',
                'Cingulum_Post_R',
                'Insula_R',
                'Hippocampus_L',
                'Hippocampus_R',
                'Amygdala_L',
                'Amygdala_R',
                'Caudate_L',
                'Caudate_R',
                'Putamen_L',
                'Putamen_R',
                'Pallidum_L',
                'Pallidum_R',
                'Thalamus_L',
                'Thalamus_R',
                'Cerebelum_Crus1_L',
                'Cerebelum_Crus1_R',
                'Cerebelum_Crus2_L',
                'Cerebelum_Crus2_R',
                'Cerebelum_3_L',
                'Cerebelum_3_R',
                'Cerebelum_4_5_L',
                'Cerebelum_4_5_R',
                'Cerebelum_6_L',
                'Cerebelum_6_R',
                'Cerebelum_7b_L',
                'Cerebelum_7b_R',
                'Cerebelum_8_L',
                'Cerebelum_8_R',
                'Cerebelum_9_L',
                'Cerebelum_9_R',
                'Cerebelum_10_L',
                'Cerebelum_10_R',
                'Vermis_1_2',
                'Vermis_3',
                'Vermis_4_5',
                'Vermis_6',
                'Vermis_7',
                'Vermis_8',
                'Vermis_9',
                'Vermis_10']

ROIs = '%0d'.join(ROI_AAL_list)
Region_AAL = ['LF',
              'LF',
              'LF',
              'LF',
              'LF',
              'LF',
              'LF',
              'LF',
              'LF',
              'LF',
              'LF',
              'LF',
              'LF',
              'LF',
              'LF',
              'LP',
              'LP',
              'LP',
              'LP',
              'LP',
              'LP',
              'LO',
              'LO',
              'LO',
              'LO',
              'LO',
              'LO',
              'LO',
              'LT',
              'LT',
              'LT',
              'LT',
              'LT',
              'LT',
              'LT',
              'LF',
              'LP',
              'LP',
              'LT',
              'RF',
              'RF',
              'RF',
              'RF',
              'RF',
              'RF',
              'RF',
              'RF',
              'RF',
              'RF',
              'RF',
              'RF',
              'RF',
              'RF',
              'RF',
              'RP',
              'RP',
              'RP',
              'RP',
              'RP',
              'RP',
              'RO',
              'RO',
              'RO',
              'RO',
              'RO',
              'RO',
              'RO',
              'RT',
              'RT',
              'RT',
              'RT',
              'RT',
              'RT',
              'RT',
              'RF',
              'RP',
              'RP',
              'RT',
              'LT',
              'RT',
              'NB',
              'NB',
              'NB',
              'NB',
              'NB',
              'NB',
              'NB',
              'NB',
              'NB',
              'NB',
              'Cer',
              'Cer',
              'Cer',
              'Cer',
              'Cer',
              'Cer',
              'Cer',
              'Cer',
              'Cer',
              'Cer',
              'Cer',
              'Cer',
              'Cer',
              'Cer',
              'Cer',
              'Cer',
              'Cer',
              'Cer',
              'Cer',
              'Cer',
              'Cer',
              'Cer',
              'Cer',
              'Cer',
              'Cer',
              'Cer']  # partition provided by PS

In [8]:
# partition into lobes - AAL
nb_ROIs_AAL = len(Region_AAL)
Lobes_AAL = ["" for x in range(len(Region_AAL))]
for kk_roi in range(len(Region_AAL)):
    Lobes_AAL[kk_roi] = regions_bst_to_lobes(Region_AAL[kk_roi])

idx_F_AAL = [i for i, j in enumerate(Lobes_AAL) if j == 'F']
idx_P_AAL = [i for i, j in enumerate(Lobes_AAL) if j == 'P']
idx_T_AAL = [i for i, j in enumerate(Lobes_AAL) if j == 'T']
idx_O_AAL = [i for i, j in enumerate(Lobes_AAL) if j == 'O']
idx_NB_AAL = [i for i, j in enumerate(Lobes_AAL) if j == 'NB']
idx_Cer_AAL = [i for i, j in enumerate(Lobes_AAL) if j == 'Cer']
idx_lobes_AAL = [idx_F_AAL, idx_P_AAL, idx_T_AAL, idx_O_AAL, idx_NB_AAL, idx_Cer_AAL]

# region_centra=Region_AAL[idx_C_AAL]
idx_lobesAAL = {"idx_lobes_AAL": idx_lobes_AAL,
                "label": "F,P,T,O,NB,Cer"}

In [11]:
idx_lobesAAL

{'idx_lobes_AAL': [[0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   35,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   74],
  [15, 16, 17, 18, 19, 20, 36, 37, 54, 55, 56, 57, 58, 59, 75, 76],
  [28, 29, 30, 31, 32, 33, 34, 38, 67, 68, 69, 70, 71, 72, 73, 77, 78, 79],
  [21, 22, 23, 24, 25, 26, 27, 60, 61, 62, 63, 64, 65, 66],
  [80, 81, 82, 83, 84, 85, 86, 87, 88, 89],
  [90,
   91,
   92,
   93,
   94,
   95,
   96,
   97,
   98,
   99,
   100,
   101,
   102,
   103,
   104,
   105,
   106,
   107,
   108,
   109,
   110,
   111,
   112,
   113,
   114,
   115]],
 'label': 'F,P,T,O,NB,Cer'}

In [9]:
savemat("idx_lobes_AAL.mat", idx_lobesAAL)
# mask
mask_AAL = Mask_Lobes_Partition_AAL(nb_ROIs_AAL, idx_F_AAL, idx_P_AAL, idx_T_AAL, idx_O_AAL, idx_NB_AAL, idx_Cer_AAL)
savemat("mask_lobes_AAL.mat", mask_AAL)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (6,) + inhomogeneous part.

In [ ]:
#%% plot connectomes
filename_mat = 'sparse_matrix'
plot_connectome_features_lobes_AAL(fig_path, db_path,filename_mat,
                                               idx_F_AAL, idx_P_AAL, idx_T_AAL, idx_O_AAL, idx_NB_AAL, idx_Cer_AAL,
                                               ROI_AAL_list, Region_AAL)